## (Visual) Chat Completion ஊகிப்பு ஆன்லைன் எண்ட்பொயின்டுகளைப் பயன்படுத்தி

இந்த எடுத்துக்காட்டு `Phi-3-vision-128k-instruct' ஐ ஊகிப்புக்காக ஒரு ஆன்லைன் எண்ட்பொயின்டிற்கு எப்படி விநியோகிப்பது என்பதைக் காட்டுகிறது.

### சுருக்கம்
* முன்-தேவைகளை அமைக்கவும்
* விரிவாக்குவதற்காக ஒரு மாதிரியை தேர்ந்தெடுக்கவும்
* ஊகிப்பிற்கான தரவுகளை பதிவிறக்கம் செய்து தயாரிக்கவும்
* நேரடி ஊகிப்பிற்காக மாதிரியை விநியோகிக்கவும்
* எண்ட்பொயின்டை பரிசோதிக்கவும்
* Azure OpenAI பாணி payload பயன்படுத்தி எண்ட்பொயின்டை சோதிக்கவும்
* வளங்களை நீக்கவும்


### 1. முன்-தேவைகள் அமைத்தல்

* தேவையான சார்புகளை நிறுவவும்
* AzureML Workspace-ஐ இணைக்கவும். மேலும் விவரங்களுக்கு [SDK அங்கீகாரத்தை அமைக்கவும்](https://learn.microsoft.com/azure/machine-learning/how-to-setup-authentication?tabs=sdk). கீழே  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` மற்றும் `<SUBSCRIPTION_ID>` மாற்றவும்.
* `azureml` சிஸ்டம் பதிவேட்டுடன் இணைக்கவும்


In [ ]:
# Import necessary modules
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)

try:
    # Try to get the default Azure credential
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # If default credential is not available, use interactive browser credential
    credential = InteractiveBrowserCredential()

try:
    # Try to create an MLClient using the provided credential
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # If MLClient creation fails, enter the details of your AML workspace manually
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<WORKSPACE_NAME>"

# Create an MLClient instance with the provided credentials and workspace details
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models, fine tuning pipelines, and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

### 2. மாதிரியை ஆன்லைன் எண்ட்பாயிண்ட் ஒன்றுக்கு வெளியிடவும்
ஆன்லைன் எண்ட்பாயிண்டுகள் மாதிரியைப் பயன்படுத்த வேண்டிய பயன்பாடுகளுடன் ஒருங்கிணைக்க பயன்படக்கூடிய நிரந்தரமான REST API-ஐ வழங்குகின்றன.


In [ ]:
# This code checks if the model with the specified name exists in the registry.
# If the model exists, it retrieves the first version of the model and prints its details.
# If the model does not exist, it prints a message indicating that the model was not found.

# model_name: Name of the model to check in the registry
model_name = "Phi-3-vision-128k-instruct"

# Get the list of versions for the specified model name
version_list = list(registry_ml_client.models.list(model_name))

# Check if any versions of the model exist in the registry
if len(version_list) == 0:
    print("Model not found in registry")
else:
    # Get the first version of the model
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    
    # Print the details of the model
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )

In [ ]:
# Import necessary modules
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = model_name[:13] + str(timestamp)
print(f"Creating online endpoint with name: {online_endpoint_name}")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=f"Online endpoint for {foundation_model.name}, for visual chat-completion task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# This code creates a deployment for the online endpoint.
# It sets the deployment name, endpoint name, model, instance type, instance count, and request settings.
# It also sets the liveness probe and readiness probe settings.
# Finally, it updates the traffic distribution for the endpoint.

"""
from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

# create a deployment
deployment_name = "phi-3-vision"
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC48ads_A100_v4",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()
"""

### 3. மாதிரி தரவுடன் endpoint ஐ சோதிக்க

நாம் கீழே உருவாக்கும் json ஐ பயன்படுத்தி மாடலுக்கு ஒரு மாதிரி கோரிக்கையை அனுப்புவோம்.


In [ ]:
# Import necessary modules
import json
import os

# Define the test JSON payload
test_json = {
    "input_data": {
        "input_string": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                        },
                    },
                    {
                        "type": "text",
                        "text": "What is shown in this image? Be extremely detailed and specific.",
                    },
                ],
            },
        ],
        "parameters": {"temperature": 0.7, "max_new_tokens": 2048},
    }
}

# Save the JSON object to a file
sample_score_file_path = os.path.join(".", "sample_chat_completions_score.json")
with open(sample_score_file_path, "w") as f:
    json.dump(test_json, f, indent=4)

# Print the input payload
print("Input payload:\n")
print(test_json)

In [ ]:
# Import necessary modules
import pandas as pd

# score the sample_chat_completions_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file=sample_score_file_path,
)
print("Raw JSON Response: \n", response, "\n")

# Parse the JSON string
json_data = json.loads(response)

# Convert the parsed JSON to a DataFrame
response_df = pd.DataFrame([json_data])
print("Generated Text:\n", response_df["output"].iloc[0])

### 4. Azure OpenAI பாணி payload ஐ பயன்படுத்தி எண்ட்பாயிண்டை சோதிக்கவும்

நாம் மாதிரி கோரிக்கையை Azure OpenAI பாணி payload உடன் மாடலுக்கு அனுப்பவுள்ளோம்.


In [ ]:
# This code defines a JSON payload for testing the online endpoint with Azure OpenAI style payload.
# It includes the model name, a list of messages with user role and content (image URL and text),
# temperature, and max_new_tokens.

aoai_test_json = {
    "model": foundation_model.name,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://www.ilankelman.org/stopsigns/australia.jpg"
                    },
                },
                {
                    "type": "text",
                    "text": "What is shown in this image? Be extremely detailed and specific.",
                },
            ],
        }
    ],
    "temperature": 0.7,
    "max_new_tokens": 2048,
}

In [ ]:
# Get the scoring uri
scoring_uri = workspace_ml_client.online_endpoints.get(
    name=online_endpoint_name
).scoring_uri
# Update the scoring uri to use for AOAI
aoai_format_scoring_uri = scoring_uri.replace("/score", "/v1/chat/completions")

# Get the key for data plane operation
data_plane_token = workspace_ml_client.online_endpoints.get_keys(
    name=online_endpoint_name
).primary_key

In [ ]:
import urllib.request
import json

# Prepare request
body = str.encode(json.dumps(aoai_test_json))
url = aoai_format_scoring_uri
api_key = data_plane_token

headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + api_key)}
req = urllib.request.Request(url, body, headers)

# Send request & get response
try:
    response = urllib.request.urlopen(req)
    result = response.read().decode("utf-8")
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

### 5. ஆன்லைன் எண்ட்பாயிண்டை நீக்க மறக்காதீர்கள்
ஆன்லைன் எண்ட்பாயிண்டை நீக்க மறக்காதீர்கள்; இல்லையெனில் அந்த எண்ட்பாயிண்ட் பயன்படுத்திய கணினி வளத்திற்கு பில்லிங் மீட்டர் ஓடிவிடும்.


In [ ]:
#Delete Workspace
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait(#)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு அறிவிப்பு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை Co-op Translator (https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக尽 முயற்சித்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கலாம் என்பதை தயவுசெய்து கவனமாகக் கொள்ளவும். இயல்பு மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பாளரின் சேவையை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பின் பயன்பாட்டினால் ஏற்படும் எந்தவொரு புரிதல் தவறுகளுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பு ஏற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
